# Student's t-test for significantly different variances in Julia:

In [1]:
using BenchmarkTools

data1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
data2 = [1.1, 2, 3, 4, 5, 6, 7, 8, 9];

Functions definition:

In [2]:
function avevar(data)
    # Given an array "data" of length "n", returns
    # its mean "ave" and its variance "var"
    n = length(data)
    ave = 0
    var = 0
    
    # Mean
    for i = 1:n
        ave =  ave + data[i]
    end
    
    ave = ave / n
    
    # Variance (of a sample population)
    for i = 1:n
        s = data[i] - ave
        var =  var + s^2
    end
        
    var = var / (n - 1)
    
    return ave, var
end;

In [3]:
function gammaln(xx)
    # Returns the value of ln[gamma(XX)] for X > 0. 
    # Full accuracy is obtained for XX > 1. 
    cof = [76.18009173, -86.50532033, 24.01409822, 
           -1.231739516, 0.120858003e-2, -0.536382e-5]
    stp = 2.50662827465
    
    x = xx - 1.0
    tmp = x + 5.5
    tmp = (x + 0.5) * log(tmp) - tmp
    ser = 1.0

    for i = 1:length(cof)
        x = x + 1
        ser = ser + cof[i] / (x)
    end

    gammaln = tmp + log(stp * ser)
    return gammaln
end;

In [4]:
function betacf(a, b, x)
    # Continued fraction for incomplete beta function, 
    # used by "betai"  
    eps = 3.0e-7
    imax = 100

    am = 1
    bm = 1
    az = 1
    qab = a + b
    qap = a + 1
    qam = a - 1
    bz = 1 - (qab * x) / qap

    # continued fraction evaluation using recurrence
    for m = 1:imax
        em = m
        tem = em + em
        d = (em * (b - m) * x) / ((qam + tem) * (a + tem))
        ap = az + d * am
        bp = bz + d * bm
        d = -((a + em) * (qab + em) * x) / ((a + tem) * (qap + tem))
        app = ap + d * az
        bpp = bp + d * bz
        aold = az
        am = ap / bpp
        bm = bp / bpp
        az = app / bpp
        bz = 1.0

        if (abs(az - aold) < eps * abs(az))
            # print('BETACF: convergence OK')
            break
        end

        if (m == imax)
            error("BETACF: a or b too big, or imax too small")
        end
    end

    betacf = az
    return betacf
end;

In [5]:
function betai(a, b, x)
    # Returns the incomplete beta function Ix(a, b)

    if ((x < 0) || (x > 1))
        error("BETAI: bad argument for x")
    end

    # Factor "bt" in front of continued fraction
    if ((x == 0 ) || (x == 1))
        bt = 0
    else
        bt = exp(gammaln(a + b) - gammaln(a) - gammaln(b) + a * log(x) + b * log(1 - x))
    end
    
    # Continued fraction
    if (x < ((a + 1) / (a + b + 2)))
        # ...directly
        betai = bt * betacf(a, b, x) / a
        return betai
    else
        # ...after symmetry transformation
        betai = 1 - bt * betacf(b, a, 1 - x) / b
        return betai
    end
end;

In [6]:
function tutest(data1, data2)
    # Given the arrays "data1" and "data2", returns the
    # Student's "t" and its significance as "prob".
    # Data are assumed to be drawn from populations with
    # the same true variance.

    # Small values of "prob" indicates that the arrays 
    # have different means.
    n1 = length(data1)
    n2 = length(data2)
    
    ave1, var1 = avevar(data1)
    ave2, var2 = avevar(data2)

    # print(ave1, ave2)
    # print(var1, var2)

    # Student's t
    t = (ave1 - ave2) / sqrt(var1/n1 + var2/n2)

    df = (var1/n1 + var2/n2)^2 / ((var1/n1)^2 / (n1 - 1) + (var2/n2)^2 / (n2 - 1))
    # significance
    prob = betai(0.5 * df, 0.5, df/(df + t^2))
    
    return t, prob
end;

### Calling the ttest function in Julia:

In [7]:
t, prob = tutest(data1, data2)

print(t, " ", prob)

-0.008635140708713388 0.9932169972438553

## Timing against Fortran:

Test function:

In [8]:
function test()
    for i = 1:1e3
        tutest(data1, data2)
        end
    end;

#### Julia:

In [9]:
@btime test()

  399.619 μs (4000 allocations: 406.25 KiB)


#### Fortran:

In [10]:
@btime run(`../fortran/fmain_tutest`);

  1.657 ms (46 allocations: 1.84 KiB)
